In [42]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
model_name = "richielo/small-e-czech-finetuned-ner-wikiann"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)
nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = """
Ačkoliv je aktuální zdražení vyšší než míra inflace, podle předsedkyně odborů pošty Jindřišky Budweiserové to není takový problém.

„Myslím si, že zdražení dopisů lidi až tolik nepálí. Tím, že je opakované, tak se o tom hodně mluví, ale když se podíváte, o kolik zdražily potraviny… Podíl poštovních služeb na výdajích domácnosti není tak velký,“ řekla Právu Budweiserová.

Problém spíše vidí ve špatném financování podniku, kdy pošta každý rok dotuje veřejné služby v řádu stamilionů korun.

„Pošta v posledních čtyřech letech provedla zásadní změny. Uspořila v ročních nákladech jednu miliardu korun, a přesto to nestačí. Bylo zrušeno více než 7000 pracovních míst, pro pošťáky to znamená velký stres, obrovskou zátěž a obavy,“ sdělila Budweiserová.

Odbory proto minulý týden vyzvaly vládu, poslance i senátory, aby situaci státního podniku řešili. Peticí je k tomu vyzvalo okolo 27 500 pracovníků.

Dnes by mělo ministerstvo vnitra oznámit výsledky výběrového řízení na nového generálního ředitele. Není přitom vyloučeno, že pozici obhájí šéf Roman Knap, který se dostal do finále výběru.

Knapa loni odvolal ministr vnitra Vít Rakušan (STAN) kvůli někdejší schůzce s lobbistou Michalem Redlem obviněným v úplatkářské kauze Dozimetr. Knap tvrdí, že se Redl na ojedinělé schůzce objevil ne­ohlášený, a nic vážného s ním tudíž ani neprobíral.

Původně měl být Knap ve funkci jen do září, ale nakonec bylo rozhodnuto, že zůstane až do výběru nástupce.

Pokud by Roman Knap zůstal, tak by to podle šéfky odborů nebyla špatná zpráva. „Změnu ve vedení v této vypjaté situaci nepovažujeme vůbec za šťastnou, v době, kdy se musí řešit transformace pošty, strategie,“ soudí Budweiserová.
"""

In [43]:
def filter_person_entities(entities: list):
    return [ent for ent in entities if 'PER' in ent['entity']]

def get_person_entities(text: str, ner_pipeline):
    entities = ner_pipeline(text)
    return filter_person_entities(entities)

def replacer(s, newstring, index, nofail=False):
    if not nofail and index not in range(len(s)):
        raise ValueError("index outside given string")

    # if not erroring, but the index is still not in the correct range..
    if index < 0:  # add it to the beginning
        return newstring + s
    if index > len(s):  # add it to the end
        return s + newstring

    # insert the new string between "slices" of the original
    return s[:index] + newstring + s[index + 1:]

def get_masked_text(text: str, ner_pipeline):
    masked_text = text
    entities = get_person_entities(example, ner_pipeline)
    for ent in entities:
        running_index = ent.get('start')
        while True:            
            masked_text = replacer(masked_text, '*', running_index)
            running_index += 1
            if running_index == ent.get('end'):
                break
    return masked_text

print('---Original---')
print(example)
print('---Masked---')
print(get_masked_text(example, nlp))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


---Original---

Ačkoliv je aktuální zdražení vyšší než míra inflace, podle předsedkyně odborů pošty Jindřišky Budweiserové to není takový problém.

„Myslím si, že zdražení dopisů lidi až tolik nepálí. Tím, že je opakované, tak se o tom hodně mluví, ale když se podíváte, o kolik zdražily potraviny… Podíl poštovních služeb na výdajích domácnosti není tak velký,“ řekla Právu Budweiserová.

Problém spíše vidí ve špatném financování podniku, kdy pošta každý rok dotuje veřejné služby v řádu stamilionů korun.

„Pošta v posledních čtyřech letech provedla zásadní změny. Uspořila v ročních nákladech jednu miliardu korun, a přesto to nestačí. Bylo zrušeno více než 7000 pracovních míst, pro pošťáky to znamená velký stres, obrovskou zátěž a obavy,“ sdělila Budweiserová.

Odbory proto minulý týden vyzvaly vládu, poslance i senátory, aby situaci státního podniku řešili. Peticí je k tomu vyzvalo okolo 27 500 pracovníků.

Dnes by mělo ministerstvo vnitra oznámit výsledky výběrového řízení na nového gen

In [46]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

class NER_masker_cz:
    
    def __init__(self, model_name="richielo/small-e-czech-finetuned-ner-wikiann"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForTokenClassification.from_pretrained(model_name)
        self.NER = pipeline("ner", model=model, tokenizer=tokenizer)
        
    def _filter_entities(self, entities: list, entity_filter: list) -> list:
        return [ent for ent in entities for ent_type in entity_filter if ent_type in ent['entity']]

    def _get_filtered_entities(self, text: str, entity_filter: list) -> list:
        entities = self.NER(text)
        return self._filter_entities(entities, entity_filter)
    
    @staticmethod
    def replacer(s, newstring, index, nofail=False):
        if not nofail and index not in range(len(s)):
            raise ValueError("index outside given string")

        # if not erroring, but the index is still not in the correct range..
        if index < 0:  # add it to the beginning
            return newstring + s
        if index > len(s):  # add it to the end
            return s + newstring

        # insert the new string between "slices" of the original
        return s[:index] + newstring + s[index + 1:]

    def get_masked_text(self, text: str, entity_filter: list = ['B-PER', 'I-PER']):
        masked_text = text
        entities = self._get_filtered_entities(example, entity_filter)
        for ent in entities:
            running_index = ent.get('start')
            while True:            
                masked_text = NER_masker_cz.replacer(masked_text, '*', running_index)
                running_index += 1
                if running_index == ent.get('end'):
                    break
        return masked_text
        

In [ ]:
masker = NER_masker_cz()
print(masker.get_masked_text(example))